In [19]:
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
import numpy as np
import os
import cv2

In [20]:
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/asl-alphabet


In [21]:
train_dir = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
test_dir = '/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'

In [22]:
IMG_SIZE = 64
NUM_CLASSES = 29
BATCH_SIZE = 32

In [23]:
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
                'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                'del', 'nothing', 'space']

In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(
    rescale=1./255
    )

In [25]:
def load_test_data(test_dir, img_size, class_labels):
    test_images = []
    test_labels = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        img = img / 255.0
        test_images.append(img)

        # Extract label from filename
        label_name = img_name.split('_')[0]
        if label_name == 'del':
            label_name = 'del'
        elif label_name == 'nothing':
            label_name = 'nothing'
        elif label_name == 'space':
            label_name = 'space'
        else:
            label_name = label_name.upper()
        label_idx = class_labels.index(label_name)
        test_labels.append(label_idx)

    test_images = np.array(test_images)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=NUM_CLASSES)
    return test_images, test_labels

In [27]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=class_labels,
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=class_labels,
    shuffle=False
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [28]:
test_images, test_labels = load_test_data(test_dir, IMG_SIZE, class_labels)

In [29]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.3),

    Conv2D(128, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.4),

    Conv2D(256, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    GlobalAveragePooling2D(),

    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
optimizer = Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

In [31]:
model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.1376 - loss: 3.5154

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 627s 284ms/step - accuracy: 0.1376 - loss: 3.5152 - val_accuracy: 0.2696 - val_loss: 2.8716 - learning_rate: 1.0000e-04
Epoch 2/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.3555 - loss: 2.5660

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 203s 93ms/step - accuracy: 0.3556 - loss: 2.5659 - val_accuracy: 0.3774 - val_loss: 2.5010 - learning_rate: 1.0000e-04
Epoch 3/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.4915 - loss: 2.0312

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 205s 94ms/step - accuracy: 0.4915 - loss: 2.0311 - val_accuracy: 0.5143 - val_loss: 2.0486 - learning_rate: 1.0000e-04
Epoch 4/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.5985 - loss: 1.6488

...

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 204s 94ms/step - accuracy: 0.9031 - loss: 0.5478 - val_accuracy: 0.7747 - val_loss: 1.0130 - learning_rate: 1.0000e-04
Epoch 19/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 204s 94ms/step - accuracy: 0.9086 - loss: 0.5285 - val_accuracy: 0.7757 - val_loss: 1.0371 - learning_rate: 1.0000e-04
Epoch 20/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 202s 93ms/step - accuracy: 0.9132 - loss: 0.5117 - val_accuracy: 0.7562 - val_loss: 1.2539 - learning_rate: 1.0000e-04
Epoch 21/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9125 - loss: 0.5123
Epoch 21: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 203s 93ms/step - accuracy: 0.9125 - loss: 0.5123 - val_accuracy: 0.7798 - val_loss: 1.0994 - learning_rate: 1.0000e-04
Epoch 22/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9279 - loss: 0.4600

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 204s 94ms/step - accuracy: 0.9279 - loss: 0.4600 - val_accuracy: 0.8164 - val_loss: 0.9236 - learning_rate: 2.0000e-05
Epoch 23/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9345 - loss: 0.4374

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 261s 93ms/step - accuracy: 0.9345 - loss: 0.4374 - val_accuracy: 0.8271 - val_loss: 0.8984 - learning_rate: 2.0000e-05
Epoch 24/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 201s 92ms/step - accuracy: 0.9374 - loss: 0.4264 - val_accuracy: 0.8178 - val_loss: 0.9229 - learning_rate: 2.0000e-05
Epoch 25/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9386 - loss: 0.4200

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 203s 93ms/step - accuracy: 0.9386 - loss: 0.4200 - val_accuracy: 0.8252 - val_loss: 0.8897 - learning_rate: 2.0000e-05
Epoch 26/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 226s 104ms/step - accuracy: 0.9390 - loss: 0.4136 - val_accuracy: 0.8240 - val_loss: 0.9230 - learning_rate: 2.0000e-05
Epoch 27/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 202s 93ms/step - accuracy: 0.9404 - loss: 0.4056 - val_accuracy: 0.8192 - val_loss: 0.9352 - learning_rate: 2.0000e-05
Epoch 28/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.9416 - loss: 0.4014
Epoch 28: ReduceLROnPlateau reducing learning rate to 3.999999898951501e-06.
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 208s 96ms/step - accuracy: 0.9416 - loss: 0.4014 - val_accuracy: 0.8221 - val_loss: 0.8900 - learning_rate: 2.0000e-05
Epoch 29/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9466 - loss: 0.3888

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 203s 93ms/step - accuracy: 0.9466 - loss: 0.3888 - val_accuracy: 0.8241 - val_loss: 0.8889 - learning_rate: 4.0000e-06
Epoch 30/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 203s 93ms/step - accuracy: 0.9447 - loss: 0.3912 - val_accuracy: 0.8255 - val_loss: 0.8981 - learning_rate: 4.0000e-06


In [32]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2857 - loss: 4.5822
Test Accuracy: 28.57%


In [34]:
model.save('asl_model.h5')

from google.colab import files
files.download('asl_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>